In [1]:
%matplotlib notebook

In [2]:
import numpy as np
import pandas as pd
import time
import sys
import os
import matplotlib.pyplot as plt
from scipy import integrate
sys.path.append(os.path.abspath("e:\\tasks\\documents_in_pku\\research\\Roaming_NN\\QCAT"))
sys.path.append(os.path.abspath("e:\\tasks\\documents_in_pku\\research\\Roaming_NN\\QCAT\\pyqcams"))
print(sys.path)
from pyqcams import qct, constants, potentials, plotters, util, analysis
import warnings
import torch
import logging
# Get the absolute path to the parent directory containing pyqcams


['e:\\tasks\\documents_in_pku\\research\\Roaming_NN\\QCAT\\SO2+', 'c:\\Users\\xumohan\\anaconda3\\envs\\py10\\python310.zip', 'c:\\Users\\xumohan\\anaconda3\\envs\\py10\\DLLs', 'c:\\Users\\xumohan\\anaconda3\\envs\\py10\\lib', 'c:\\Users\\xumohan\\anaconda3\\envs\\py10', '', 'c:\\Users\\xumohan\\anaconda3\\envs\\py10\\lib\\site-packages', 'c:\\Users\\xumohan\\anaconda3\\envs\\py10\\lib\\site-packages\\win32', 'c:\\Users\\xumohan\\anaconda3\\envs\\py10\\lib\\site-packages\\win32\\lib', 'c:\\Users\\xumohan\\anaconda3\\envs\\py10\\lib\\site-packages\\Pythonwin', 'e:\\tasks\\documents_in_pku\\research\\Roaming_NN\\QCAT', 'e:\\tasks\\documents_in_pku\\research\\Roaming_NN\\QCAT\\pyqcams']


In [3]:
# Paths to the MLP model and scaler
model_path = r'E:\tasks\documents_in_pku\research\Roaming_NN\QCAT\NN\results\best_model.pth'   

# 定义质量单位
m1 = constants.mass_O * constants.u2me  # Oxygen
m2 = constants.mass_O * constants.u2me  # Oxygen
m3 = constants.mass_S * constants.u2me  # Sulfur

E0 = 40000 * constants.K2Har # Collision energy (K)
b0 = 0      # Impact parameter
R0 = 50     # Initial distance in Bohr

# 加载已训练的模型
input_dim = 3
neuron = 64
process_param_l = 1.5  # 确保与训练时一致
# Load MLP model
model = potentials.load_MLP_model(model_path, input_dim, neuron,process_param_l)


# Run one trajectory
## Input Dictionary
This is an example input for the reaction `H$_2$ + Ca $\rightarrow$ CaH + H`, where `H$_2$` is initialized in the v=3, j=3 state. The dictionary can be imported from the `inputs.py` file, or written directly as shown here.

First, create the QCT calculation input dictionary, which requires masses `mi`, collision energy `E0`, impact parameter `b0`, initial distance `R0`, all three `Molecule` objects `mol_ij`, 3-body potential `vt` and partial derivatives `dvtdrij`, and integration parameters `integ`. 

The `Molecule` object requires the respective masses `mi, mj`, diatiomic potential `Vij` and its derivative `dVij`, and range `xmin`,`xmax`. The molecule formed between atoms 1 and 2 is the initial molecule, thus needs initial rovibrational state (`vi`,`ji`), and number of DVR points `npts`. If the vibrational energy spectrum is known, `vi` and `npts` can be replaced by the keyword argument `Ei` in Hartree. 

In [4]:
# 初始化 Molecule 实例，传递模型而不是 Vij 和 dVij
v12,dv12=potentials.morse(de = 6.08/constants.Har2ev, re = 1.40104284795, alpha = 2.97) # O2
v23,dv23=potentials.morse(de = 6.41/constants.Har2ev, re = 1.481, alpha = 6.54) # SO
v31,dv31=potentials.morse(de = 6.41/constants.Har2ev, re = 1.481, alpha = 6.54) # SO
mol12 = qct.Molecule(
    potential_type='analytical',
    mi=m1,
    mj=m2,
    vi=3,
    ji=3,
    Vij = v12, dVij = dv12,
    Ei=None,  # Set if initial energy is known
    xmin=0.5,
    xmax=30,
    npts=1000
)  # O2+

mol23 = qct.Molecule(
    potential_type='analytical',
    mi=m2,
    mj=m3,
    Vij = v23, dVij = dv23,
    xmin=0.5,
    xmax=30,
    npts=1000
)  # SO+

mol31 = qct.Molecule(
    potential_type='analytical',
    mi=m3,
    mj=m1,
    Vij = v31, dVij = dv31,
    Ei=None,  # Set if initial energy is known
    xmin=0.5,
    xmax=30,
    npts=1000
)  # SO+

v123, dv123dr12, dv123dr23, dv123dr31 = potentials.mlp_potential_function(model) # SO2+

input_dict = {
    'm1': m1, 'm2': m2, 'm3': m3,
    'E0': E0, 'b0': b0, 'R0': R0, 'seed': None,
    'mol_12': mol12, 'mol_23': mol23, 'mol_31': mol31,
    'vt': v123, 'dvtdr12': dv123dr12, 'dvtdr23': dv123dr23, 'dvtdr31': dv123dr31,
    'integ': {'t_stop': 4, 'r_stop': 2, 'r_tol': 1e-10, 'a_tol':1e-8, 'econs':1e-5, 'lcons':1e-5}
}

## Study Potential Energy Curves
Before we run the trajectory, we should examine the potential functions describing the 3 potential molecules. We can calculate the energy spectrum of H2 using `Molecules.DVR`, and we can also plot the energy level.

In [5]:
mol12.DVR()  # Energy spectrum
mol12.turningPts(initial=True)  # Set classical turning points

1
1
1
1


In [6]:
# Define r ranges
r12 = np.linspace(mol12.xmin, mol12.xmax, mol12.npts)
r23 = np.linspace(mol23.xmin, mol23.xmax, 1000)
r31 = np.linspace(mol31.xmin, mol31.xmax, 1000)
# Plot PES
plt.figure(figsize=(8,6))
plt.plot(r12, mol12.Veff(r12), label=r'V$_{H_2}$')
plt.hlines(mol12.Ei, mol12.rm, mol12.rp, colors='blue', linestyles='dashed', label=r'$E_i$')
plt.plot(r23, mol23.Vij(r23), label=r'V$_{CaH}$')
plt.plot(r31, mol31.Vij(r31), label=r'V$_{CaH}$')
plt.xlabel(r'r [a$_0$]')
plt.ylabel(r'V [E$_H$]')
plt.title('Potential Energy Surfaces')
plt.legend()
plt.savefig('results/PES.png')
plt.show()

<IPython.core.display.Javascript object>

We can run a trajectory and obtain an output by initializing a `Trajectory` object and calling `runT`. Then we can use the plotter tool to plot the trajectory in 2d and 3d. 

In [7]:
# Run a single trajectory
traj = qct.Trajectory(**input_dict)
traj.runT()

# Plot the trajectory in 2D
plt.figure(figsize=(8,6))
plotters.traj_plt(traj)
plt.title('Trajectory Plot')
plt.savefig('results/QCT_2D.png')
plt.show()

# Plot the trajectory in 3D
plt.figure(figsize=(8,6))
plotters.traj_3d(traj)
plt.title('Trajectory 3D Plot')
plt.savefig('results/QCT_3D.png')
plt.show()

# Get and print results
attrs = ('delta_e', 'delta_l')
results = util.get_results(traj, *attrs)
print("Single Trajectory Results:", results)


1
1
1


KeyboardInterrupt: 

We can inspect the results using `util.get_results`, including any extra attributes such as `delta_e` and `delta_l`. 

In [ ]:
attrs = ('delta_e','delta_l')
print(util.get_results(traj, *attrs))

## Plotting all products
The trajectory can end with three possibilities:
1. Quenching: Ca + H$_2$ $\rightarrow$ Ca + H$_2$
2. Reaction: Ca + H$_2$ $\rightarrow$ CaH + H
3. Dissociation: Ca + H$_2$ $\rightarrow$ Ca + H + H

In [ ]:
# Run and filter trajectories for quenching
quench = qct.Trajectory(**input_dict)
quench.runT()
n = 0
while (quench.count.get('dissociation', 0) == 0) and n < 100:  # Adjust key as per 'dissociation_channels'
    quench.runT()
    n += 1
print(f"Quenching Trajectory Attempts: {n}, Counts: {quench.count}")

In [ ]:
# Run and filter trajectories for reaction
react = qct.Trajectory(**input_dict)
react.runT()
n = 0
while (react.count.get('SO+', 0) == 0 and react.count.get('O', 0) == 0) and n < 100:
    react.runT()
    n += 1
print(f"Reaction Trajectory Attempts: {n}, Counts: {react.count}")

In [ ]:
# Run and filter trajectories for dissociation
diss = qct.Trajectory(**input_dict)
diss.runT()
n = 0
while (diss.count.get('dissociation', 0) == 0) and n < 100:
    diss.runT()
    n += 1
print(f"Dissociation Trajectory Attempts: {n}, Counts: {diss.count}")

In [ ]:
# Plot all products
fig, ax = plt.subplots(1, 3, sharey=True, figsize=(15,5))
plotters.traj_plt(diss, title=False, legend=True, ax=ax[0])
plotters.traj_plt(react, title=False, legend=False, ax=ax[1])
plotters.traj_plt(quench, title=False, legend=False, ax=ax[2])
plt.subplots_adjust(wspace=0, bottom=0.15)
for i, subplot in enumerate(ax):
    subplot.set_xlabel('t (ps)')
ax[1].set_ylabel('')
ax[2].set_ylabel('')
plt.savefig('results/all_products.png')
plt.show()

## Parallel Run
For the same initial state, use the `runN` method to run `nTraj` number of trajectories using all available CPUs. This can be controlled with the `cpus` argument. You can save to short and long output files, and request `Trajectory` attributes with argument `attrs` to save them to the long output. This can be done directly in the input file. 

In [ ]:
# Run multiple trajectories
nTraj = 100  # Adjust as needed
full_results, counts = qct.runN(
    nTraj, 
    input_dict, 
    cpus=8, 
    short_out='short_output.csv', 
    long_out='long_output.csv'
)
print("Multiple Trajectories Counts:", counts)

# Perform analysis
opacity_df = analysis.opacity(
    'long_output.csv', 
    GB=True, 
    vib=True, 
    rot=True, 
    output='opacity.csv'
)
cross_section_df = analysis.crossSection(
    opacity_df, 
    GB=True, 
    vib=True, 
    rot=True, 
    output='cross_section.csv'
)
rate_coefficients_df = analysis.rate(
    cross_section_df, 
    mu=1.0, 
    GB=True, 
    vib=True, 
    rot=True, 
    output='rate_coefficients.csv'
)

# Visualize cross sections
plt.figure(figsize=(8,6))
plt.errorbar(
    cross_section_df['e'], 
    cross_section_df['sig_R'], 
    yerr=cross_section_df['sig_Rerr'], 
    label='Reaction Cross Section'
)
plt.errorbar(
    cross_section_df['e'], 
    cross_section_df['sig_Q'], 
    yerr=cross_section_df['sig_Qerr'], 
    label='Quenching Cross Section'
)
plt.errorbar(
    cross_section_df['e'], 
    cross_section_df['sig_Diss'], 
    yerr=cross_section_df['sig_Diss_err'], 
    label='Dissociation Cross Section'
)
plt.xlabel('Energy (K)')
plt.ylabel('Cross Section (cm²)')
plt.title('Cross Sections vs Energy')
plt.legend()
plt.savefig('results/Cross_Sections.png')
plt.show()

# Visualize rate coefficients
plt.figure(figsize=(8,6))
plt.errorbar(
    rate_coefficients_df['e'], 
    rate_coefficients_df['k_R'], 
    yerr=rate_coefficients_df['k_Rerr'], 
    label='Reaction Rate Coefficient'
)
plt.errorbar(
    rate_coefficients_df['e'], 
    rate_coefficients_df['k_Q'], 
    yerr=rate_coefficients_df['k_Qerr'], 
    label='Quenching Rate Coefficient'
)
plt.errorbar(
    rate_coefficients_df['e'], 
    rate_coefficients_df['k_Diss'], 
    yerr=rate_coefficients_df['k_Diss_err'], 
    label='Dissociation Rate Coefficient'
)
plt.xlabel('Energy (K)')
plt.ylabel('Rate Coefficient (cm³/s)')
plt.title('Rate Coefficients vs Energy')
plt.legend()
plt.savefig('results/Rate_Coefficients.png')
plt.show()


## Analysis
Use the provided analysis functions for the state-specific opacity,cross section, and rate coefficients of the reaction. The columns are indexed according to the final outcome, where `R` represents a reaction, `Q` represents quenching, and `Diss` represents dissociation. The error is also included for each outcome. In this example we use Gaussian Binning, and look for the vibrational state-specific results from H$_2$ initialized in the (v=0,j=0) state. 

In [ ]:
mu = m3*(m1+m2)/(m1+m2+m3)
# Paths to simulation results
long_output_path = 'results/long_output.csv'  # Adjust if different
short_output_path = 'results/short_output.csv'  # Adjust if different

# Calculate Opacity
opacity = analysis.opacity(long_output_path, GB=True, vib=True, rot=False)

# Calculate Cross Section
sigma = analysis.crossSection(long_output_path, GB=True, vib=True, rot=False)

# Calculate Rate Coefficient
rate = analysis.rate(long_output_path, mu=mu, GB=True, vib=True, rot=False)


In [ ]:
opacity.head()

Check that the opacity function is continuous, with a clearly defined maximum impact parameter. Here we sum over the final v states to get the full opacity at a given collision energy.

In [ ]:
# Reset index if necessary
o = opacity.reset_index()

# Group by vibrational state, rotational state, energy, and impact parameter, then sum pR
pR = o.groupby(['vi', 'ji', 'e', 'b'])['pR'].sum().reset_index()

# Plot pR vs b for each energy level
plt.figure(figsize=(8,6))
for ev in pR['e'].unique():
    pR_e = pR[pR['e'] == ev]
    plt.plot(pR_e['b'], pR_e['pR'], '.', label=f'E = {ev} K')
plt.xlabel('Impact Parameter (b)')
plt.ylabel('Probability of Reaction (pR)')
plt.title('Reaction Probability vs Impact Parameter for Different Energies')
plt.legend()
plt.grid(True)
plt.savefig('results/Reaction_Probability_vs_Impact_Parameter.png')
plt.show()

Since we kept track of the final vibrational states, we can look into the distribution of CaH(v$_f$,j).

In [ ]:
# Reset index to make 'e' a column if it's part of the index
if 'e' in rate.index.names:
    r = rate.reset_index('e')
else:
    r = rate.copy()

# Plot rate coefficients vs vibrational state for each energy
plt.figure(figsize=(8,6))
for ev in r['e'].unique():
    re = r[r['e'] == ev]
    plt.errorbar(re['v'], re['k_R'], re['k_Rerr'], fmt='o', capsize=3, label=f'E = {ev:.1f} K')
plt.xlabel('Final Vibrational State (v_f)')
plt.ylabel(r'Rate Coefficient $k_R$ [$\mathrm{cm^3/s}$]')
plt.title('Rate Coefficients vs Final Vibrational State for Different Energies')
plt.legend()
plt.grid(True)
plt.savefig('results/Rate_Coefficients_vs_Vibrational_State.png')
plt.show()

To inspect the role of initial vibrational state of H2 in the reaction, we calculate general rates (not state-specific). Here we use a previously calculated opacity file that varies with initial vibrational state.

In [ ]:
opacity_file_path = 'results/opacity.txt'  # Adjust if different

# Check if the file exists
if os.path.exists(opacity_file_path):
    opacity_reloaded = pd.read_csv(opacity_file_path)
    opacity_reloaded = opacity_reloaded.set_index(['vi', 'ji', 'e', 'b'])
    
    # Recalculate rate coefficients
    rates = analysis.rate(opacity_reloaded, mu=mu).reset_index()
    
    # Filter for positive reaction rates
    rates = rates[rates['k_R'] > 0]
    
    # Plot rate coefficients vs energy for each initial vibrational state
    plt.figure(figsize=(8,6))
    for v in rates['vi'].unique():
        r = rates[rates['vi'] == v]
        plt.errorbar(r['e'], r['k_R'], r['k_Rerr'], fmt='.--', capsize=3, label=f'vi = {v}')
    plt.xscale('log')
    plt.yscale('log')
    plt.xlabel('Energy (K)')
    plt.ylabel(r'Rate Coefficient $k_R$ [$\mathrm{cm^3/s}$]')
    plt.title('Rate Coefficients vs Energy for Different Initial Vibrational States')
    plt.legend()
    plt.grid(True, which="both", ls="--")
    plt.tick_params(which='both', direction='in')
    plt.savefig('results/Rate_Coefficients_vs_Energy.png')
    plt.show()
else:
    print(f"Opacity file '{opacity_file_path}' not found. Skipping this step.")